<a href="https://colab.research.google.com/github/jhk0530/spider/blob/master/72_CNN%EC%9C%BC%EB%A1%9C_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN
neural network의 단점 층이 늘어나면 구림을 극복한 문제

caltech 101 data의 chair, camera, butterfly, elephant, flamingo를 분류

이미지가 각각 전부 자기 맘대로라서, 획일화 필요.

런타임 디렉토리에 각 폴더를 만들고, 사진 데이터를 업로드 함

In [14]:
from PIL import Image
import os,glob
import numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = '.' 
categories = ['chair', 'camera', 'butterfly', 'elephant', 'flamingo']

nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_w*image_h*3


X = []
Y = []

for idx, cat in enumerate(categories):
  label = [0 for i in range(nb_classes)]
  label[idx] = 1

  image_dir = caltech_dir + '/' + cat  
  files = glob.glob(image_dir+'/*.jpg')  
  for i,f in enumerate(files):
    img = Image.open(f) 
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))

    data = np.asarray(img)
    X.append(data)
    Y.append(label)
    
X = np.array(X)      
Y = np.array(Y)

print(len(X))
print(len(Y))

X_train ,X_test, y_train, y_test = train_test_split(X, Y)

xy = (X_train, X_test, y_train, y_test)

np.save("./5obj.npy", xy)

print("ok,", len(Y))




334
334
ok, 334


|카테고리 이름 | 레이블 |
|:---:|:---:|
|chair|[1,0,0,0,0]|
|camera|[0,1,0,0,0]|
|butterfly|[0,0,1,0,0]|
|elephant|[0,0,0,1,0]|
|flamingo|[0,0,0,0,1]|

`.glob()` 함수를 이용하여, `.jpg`파일만 검색.

`.npy`는 `.RData`랑 비슷한 내용인 것 같다.


In [22]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np

categories = ['chair', 'camera', 'butterfly', 'elephant', 'flamingo']

nb_classes = len(categories)

image_w = 64
image_h = 64

# print(len(X_train))
# 왠지 모르겠는데 실행이 안되고, 다행히도 이미 데이터는 로드 되어있으므로 패스
# X_train, X_test, y_train , y_test = np.load('./5obj.npy')

X_train = X_train.astype("float")/256
X_test = X_test.astype("float")/256
print("X_train shape : ", X_train.shape)

# 모델 구축

model = Sequential()

# 1st layer

model.add(Convolution2D(32, 3, 3 , border_mode = 'same', input_shape = X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2nd layer 

model.add(Convolution2D(64,3,3, border_mode = 'same'))
model.add(Activation('relu'))

# 3rd layer ?

model.add(Convolution2D(64,3,3))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation("softmax"))


model.compile(loss = 'binary_crossentropy', optimizer  ='rmsprop', metrics =['accuracy'])

model.fit(X_train, y_train, batch_size = 32, epochs = 50)

score = model.evaluate(X_test, y_test)

print('loss = ', score[0])
print('accuracy = ', score[1])



X_train shape :  (250, 64, 64, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


Epoch 1/50
250/250 [==============================] - 4s 15ms/step - loss: 0.8842 - accuracy: 0.7432
Epoch 2/50
250/250 [==============================] - 3s 13ms/step - loss: 0.4793 - accuracy: 0.8024
Epoch 3/50
250/250 [==============================] - 3s 13ms/step - loss: 0.4754 - accuracy: 0.8032
Epoch 4/50
250/250 [==============================] - 3s 13ms/step - loss: 0.4508 - accuracy: 0.8040
Epoch 5/50
250/250 [==============================] - 3s 13ms/step - loss: 0.4025 - accuracy: 0.8224
Epoch 6/50
250/250 [==============================] - 3s 13ms/step - loss: 0.3994 - accuracy: 0.8312
Epoch 7/50
250/250 [==============================] - 3s 13ms/step - loss: 0.3168 - accuracy: 0.8656
Epoch 8/50
250/250 [==============================] - 3s 13ms/step - loss: 0.2780 - accuracy: 0.8752
Epoch 9/50
250/250 [==============================] - 3s 13ms/step - loss: 0.2069 - accuracy: 0.9168
Epoch 10/50
250/250 [==============================] - 3s 13ms/step - loss: 0.1731 - accura


위 코드 대로 돌렸더니, train 데이터에 한해서 loss = 2.23e-4, accuracy = 1이 나오고 test에는 loss 0.69, accuracy = 0.87이 나왔다 개쩐다 진짜

cnn 모델 형태는 `합성곱`, `relu`, `max pooling` 을 3개씩 쌓은 모델.

왜 이렇게 설계 했는지가 중요 한 것 같은데 그런 설명은 안했네 해보니까 되더라 이런건가

실행과정에서
`border_mode`가 `padding`으로 업데이트 되었다는 메시지가 뜨긴 뜸.
별로 중요한 것은 아니니까 패스.

```python
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
```

중요한점은, 
- input 형태를 맞춰주는 것과 
- output 데이터 카테고리 수를 지정하는 것

```python
model.add(Convolution2D(32,3,3, border_mode = 'same', input_shape = X_train.shape[1:]) # input 형태 지정

model.add(Dense(nb_classes)) # 카테고리 지정
```

정확성을 높이기 위한 좋은 방법 중 하나는 데이터를 더 넣는 것.

- 노이즈 추가
- 이미지 각도 변경
- 이미지 반전
- 확대 축소
- normalize
- 명암대비, 감마값 수정 등등.


In [27]:
pre = model.predict(X_test)

for i, v in enumerate(pre):
  pre_ans = v.argmax() # 예측한 값
  ans = y_test[i].argmax() # 실제 값
  dat = X_test[i] 

  if ans == pre_ans : continue

  print("[NG]", categories[pre_ans], '!=', categories[ans])
  print(v)

  # error directory 만들어줘야 함

  fname = 'error/'+  str(i) + '-' + categories[pre_ans] + '-ne-' + categories[ans] + '.png'

  dat *= 256
  img = Image.fromarray(np.uint8(dat))
  img.save(fname)




[NG] flamingo != camera
[0. 0. 0. 0. 1.]
[NG] butterfly != flamingo
[6.1441919e-05 8.5617836e-07 6.9915718e-01 1.0829366e-06 3.0077946e-01]
[NG] elephant != butterfly
[6.0814717e-03 4.1381600e-07 1.5752497e-05 7.9451352e-01 1.9938886e-01]
[NG] elephant != butterfly
[3.3815520e-06 1.7682903e-07 3.5980016e-02 9.6401536e-01 1.0584405e-06]
[NG] flamingo != chair
[3.4459212e-04 5.8089971e-04 1.2586547e-04 4.2705378e-05 9.9890590e-01]
[NG] chair != elephant
[8.628113e-01 7.370903e-06 9.157967e-04 9.966412e-02 3.660135e-02]
[NG] butterfly != elephant
[8.1977333e-07 2.3644270e-10 9.9999845e-01 2.8127177e-08 6.7946684e-07]
[NG] butterfly != chair
[1.1423084e-01 2.6767208e-07 8.8576144e-01 7.4477448e-06 4.1232600e-09]
[NG] chair != butterfly
[9.9724233e-01 2.3444546e-08 2.5270469e-03 1.3366011e-04 9.6956450e-05]
[NG] flamingo != elephant
[0.003042   0.00129875 0.06044317 0.04876222 0.88645387]
[NG] flamingo != chair
[2.0016685e-07 2.9459283e-02 3.6482117e-05 1.5767806e-07 9.7050381e-01]
[NG] cha

학습한 모델 저장하기 

모델 저장은 `hdf5`형식의 데이터를 다루는 `h5py`모듈을 사용함.....?



In [28]:
hdf5_file = '5obj-model.hdf5'

if os.path.exists(hdf5_file):
  model.load_weights(hdf5_file)
else :
  model.fit(X_train, y_train, batch_size = 32, epochs = 50)
  model.save_weights(hdf5_file)


Epoch 1/50
250/250 [==============================] - 3s 13ms/step - loss: 0.4217 - accuracy: 0.9384
Epoch 2/50
250/250 [==============================] - 3s 14ms/step - loss: 0.1536 - accuracy: 0.9840
Epoch 3/50
250/250 [==============================] - 3s 13ms/step - loss: 0.0110 - accuracy: 0.9968
Epoch 4/50
250/250 [==============================] - 3s 13ms/step - loss: 3.3534e-05 - accuracy: 1.0000
Epoch 5/50
250/250 [==============================] - 3s 14ms/step - loss: 6.2378e-04 - accuracy: 1.0000
Epoch 6/50
250/250 [==============================] - 3s 13ms/step - loss: 1.5200e-04 - accuracy: 1.0000
Epoch 7/50
250/250 [==============================] - 3s 13ms/step - loss: 9.8712e-05 - accuracy: 1.0000
Epoch 8/50
250/250 [==============================] - 3s 13ms/step - loss: 9.5447e-05 - accuracy: 1.0000
Epoch 9/50
250/250 [==============================] - 3s 13ms/step - loss: 7.1651e-05 - accuracy: 1.0000
Epoch 10/50
250/250 [==============================] - 3s 14ms/step

JSON이나 YAML 형태로 출력도 가능 하다고 한다.